In [3]:
import pandas as pd
import glob
import os

In [4]:
AIS_dir = "/home/imonge/Documents/AIS"
output_dir = "../../../../data/test_correlation/AIS/AIS.pkl"

In [ ]:
#### Data Preparation ####

In [5]:
## Retrieve csv files
csv_files = glob.glob(os.path.join(AIS_dir, "*.csv"))
dataframes = []

# Load each file as a DataFrame
for file in csv_files:
    try:
        df = pd.read_csv(file, parse_dates=["datetime"])
        dataframes.append(df)
    except Exception as e:
        print(f"Erreur lors de la lecture de {file} : {e}")

# Merge csv
AIS = pd.concat(dataframes, ignore_index=True)

# Standardise columns names
AIS.rename(columns=lambda x: x.strip().lower(), inplace=True)

# Convert 0 for width and length and draught in NaN
AIS["length"] = AIS["length"].replace(0, pd.NA)
AIS["width"] = AIS["width"].replace(0, pd.NA)
AIS["draught"] = AIS["draught"].replace(0, pd.NA)

/tmp/ipykernel_63363/3078811834.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file, parse_dates=["datetime"])
/tmp/ipykernel_63363/3078811834.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  AIS = pd.concat(dataframes, ignore_index=True)


In [6]:
## Remove incoherent data
# Datetime duplicates
AIS.drop_duplicates(subset=["mmsi", "datetime"], inplace=True)

# Remove several lengths or widths for one mmsi
mmsi_multi_length = AIS.groupby("mmsi")["length"].nunique()
mmsi_multi_width = AIS.groupby("mmsi")["width"].nunique()

mmsi_invalid = mmsi_multi_length[mmsi_multi_length > 1].index.union(
    mmsi_multi_width[mmsi_multi_width > 1].index
)

AIS = AIS[~AIS["mmsi"].isin(mmsi_invalid)].copy()

In [7]:
## NaN values
# Replace NaN in length, width and draught columns by the known values for each mmsi
mmsi_length_map = AIS.groupby("mmsi")["length"].first()
mmsi_width_map = AIS.groupby("mmsi")["width"].first()
mmsi_draught_map = AIS.groupby("mmsi")["draught"].first()

AIS["length"] = AIS.apply(
    lambda row: mmsi_length_map[row["mmsi"]] if pd.isna(row["length"]) else row["length"], axis=1
)
AIS["width"] = AIS.apply(
    lambda row: mmsi_width_map[row["mmsi"]] if pd.isna(row["width"]) else row["width"], axis=1
)
AIS["draught"] = AIS.apply(
    lambda row: mmsi_draught_map[row["mmsi"]] if pd.isna(row["draught"]) else row["draught"], axis=1
)

# Remove mmsi with no values for length or width
mmsi_with_length = AIS.groupby("mmsi")["length"].apply(lambda x: x.notna().any())
mmsi_with_width = AIS.groupby("mmsi")["width"].apply(lambda x: x.notna().any())
valid_mmsi = mmsi_with_length & mmsi_with_width
AIS = AIS[AIS["mmsi"].isin(valid_mmsi[valid_mmsi].index)].copy()

In [8]:
## Save dataset as pkl
# Sort by datetime
AIS.sort_values(by="datetime", inplace=True)

# Reset index
AIS.reset_index(drop=True, inplace=True)

# Save dataframe as pkl
AIS.to_pickle(output_dir)

In [ ]:
#### Data Verification ####

In [9]:
# AIS dataframe verification
# Datetime duplicates
duplicate_mmsi_datetime = AIS.duplicated(subset=["mmsi", "datetime"]).sum()
print(f"Duplicates mmsi + datetime: {duplicate_mmsi_datetime}")

# Mmsi with different length / width
nb_multi_length = AIS.groupby("mmsi")["length"].nunique()
nb_multi_length = nb_multi_length[nb_multi_length > 1].index

nb_multi_width = AIS.groupby("mmsi")["width"].nunique()
nb_multi_width = nb_multi_width[nb_multi_width > 1].index

nb_multi_both = nb_multi_length.intersection(nb_multi_width)

print(f"mmsi with multiple length: {len(nb_multi_length)}")
print(f"mmsi with multiple width: {len(nb_multi_width)}")
print(f"mmsi with multiple both: {len(nb_multi_both)}")


Duplicates mmsi + datetime: 0
mmsi with multiple length: 0
mmsi with multiple width: 0
mmsi with multiple both: 0


In [10]:
# Nan values
# Number of nan values
total_nan =  AIS.isna().sum().sum()
print(f"Total nan values: {total_nan}")

# Columns corresponding
na_per_column = AIS.isna().sum()
print("Columns corresponding")
print(f"{na_per_column}")

# Number of boats without dimension
mmsi_no_dim = AIS[AIS["length"].isna() & AIS["width"].isna()]["mmsi"].unique()
print(f"Number of boats without dimension: {len(mmsi_no_dim)}")

Total nan values: 215109
Columns corresponding
mmsi             0
datetime         0
lon              0
lat              0
sog              0
cog              0
draught     215109
length           0
width            0
dtype: int64
Number of boats without dimension: 0


In [11]:
length_counts = AIS.groupby("mmsi")["length"].nunique()
width_counts = AIS.groupby("mmsi")["width"].nunique()

mmsi_valid = length_counts[(length_counts == 1) & (width_counts == 1)].index
mmsi_invalid = length_counts[(length_counts > 1) & (width_counts > 1)].index
mmsi_missing = length_counts[length_counts == 0].index
tot_boat = AIS["mmsi"].nunique()
tot_mmsi = len(mmsi_valid) + len(mmsi_invalid) + len(mmsi_missing)

print(f"mmsi avec une seule valeur de longueur et largeur: {len(mmsi_valid)}")
print(f"mmsi avec plusieurs valeurs de longueurs ou largeurs: {len(mmsi_invalid)}")
print(f"mmsi sans valeurs renseignées: {len(mmsi_missing)}")
print(f"Total number of boats (mmsi): {tot_boat}")
print(tot_mmsi)

mmsi avec une seule valeur de longueur et largeur: 6352
mmsi avec plusieurs valeurs de longueurs ou largeurs: 0
mmsi sans valeurs renseignées: 0
Total number of boats (mmsi): 6352
6352


In [19]:
# csv dataset : mmsi without length or width
out_path = "../../../archives/AIS/df_no_dim_2.csv"

df_no_dim = AIS[AIS["length"].isna() & AIS["width"].isna()]

df_mmsi_no_dim = df_no_dim[["mmsi", "sog", "length", "width", "draught"]]

pd.DataFrame(df_mmsi_no_dim).to_csv(out_path, index=False)

# df_no_dim.to_csv(out_path, index=False)

In [ ]:
out_path = "../../../archives/AIS/mmsi_no_dim_csv.csv"
mmsi_no_dim_df = AIS[AIS["length"].isna() & AIS["width"].isna()]
mmsi_no_dim_df.to_csv(out_path, index=False)